In [ ]:
from pptx import Presentation
from pptx.util import Pt, Inches
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
import textwrap
import logging

logging.basicConfig(level=logging.INFO)

class SimplifiedContentGenerator:
    def __init__(self, template_path: str):
        self.prs = Presentation(template_path)
        self._validate_template()
        self.LINE_HEIGHT = Pt(9)
        self.CHARS_PER_ROW = 55
        self.WHITE = RGBColor(255, 255, 255)
        
    def _remove_unused_slides(self, slide_indices):
        """Remove unused slides using direct XML manipulation"""
        xml_slides = self.prs.slides._sldIdLst  # Access XML element list
        
        for slide_idx in sorted(slide_indices, reverse=True):
            if slide_idx >= len(xml_slides):
                continue
                
            # Get XML slide element and its relationship ID
            slide_element = xml_slides[slide_idx]
            rId = slide_element.rId  # Correct rId access from XML element
            
            # Remove relationship and XML element
            self.prs.part.drop_rel(rId)
            xml_slides.remove(slide_element)
            logging.info(f"Removed slide {slide_idx + 1}")
        
    def _validate_template(self):
        """Modified template validation without slide iteration"""
        required_shapes = {
            0: ["coSummaryShape"],
            1: ["coSummaryShape"] if len(self.prs.slides) > 1 else []
        }
        
        # Check first 2 slides using index access
        for slide_idx in [0, 1]:
            if slide_idx >= len(self.prs.slides):
                break
            slide = self.prs.slides[slide_idx]
            if required_shapes.get(slide_idx):
                if not any(s.name == "coSummaryShape" for s in slide.shapes):
                    raise ValueError(f"Missing coSummaryShape on slide {slide_idx+1}")
    def parse_markdown(self, md_content: str) -> list:
        cleaned_lines = []
        for line in md_content.strip().split('\n'):
            clean_line = line.strip().lstrip('#*- ').replace('**', '').replace('__', '')
            if clean_line:
                wrapper = textwrap.TextWrapper(width=self.CHARS_PER_ROW)
                cleaned_lines.extend(wrapper.wrap(clean_line))
        return cleaned_lines

    def _format_paragraph(self, paragraph, text, is_continuation=False):
        paragraph.paragraph_format.left_indent = Inches(0.07)
        paragraph.paragraph_format.space_before = Pt(0)
        paragraph.paragraph_format.space_after = Pt(0)
        paragraph.paragraph_format.line_spacing = 1.0
        
        run = paragraph.add_run()
        run.text = f"(continued) {text}" if is_continuation else text
        run.font.name = 'Arial'
        run.font.bold = True
        run.font.size = Pt(9)
        run.font.color.rgb = self.WHITE

    def generate(self, md_content: str, output_path: str):
        try:
            lines = self.parse_markdown(md_content)
            current_slide = 0
            line_idx = 0
            
            while line_idx < len(lines) and current_slide < 2:
                slide = self.prs.slides[current_slide]
                shape = next(s for s in slide.shapes if s.name == "coSummaryShape")
                tf = shape.text_frame
                tf.clear()
                
                para = tf.paragraphs[0]
                remaining_space = self._calculate_max_lines(shape)
                
                while line_idx < len(lines) and remaining_space > 0:
                    if line_idx > 0 and remaining_space == self._calculate_max_lines(shape):
                        self._format_paragraph(para, lines[line_idx], is_continuation=True)
                    else:
                        self._format_paragraph(para, lines[line_idx])
                    
                    line_idx += 1
                    remaining_space -= 1
                    if remaining_space > 0:
                        para = tf.add_paragraph()
                
                current_slide += 1
            
            self._remove_excess_slides(current_slide)
            self.prs.save(output_path)
            logging.info(f"Generated {current_slide} slides successfully")
            
        except Exception as e:
            logging.error(f"Generation failed: {str(e)}")
            raise

    def _calculate_max_lines(self, shape):
        return int(shape.height / self.LINE_HEIGHT) - 1

    def _remove_excess_slides(self, used_slides):
        for slide_idx in range(len(self.prs.slides)-1, used_slides-1, -1):
            xml_slides = self.prs.slides._sldIdLst
            slide_element = xml_slides[slide_idx]
            self.prs.part.drop_rel(slide_element.rId)
            xml_slides.remove(slide_element)


# Sample markdown content for testing
sample_md_content = """
# Executive Summary
This comprehensive analysis provides key insights into the current market conditions and strategic recommendations for the upcoming fiscal quarter.

## Key Performance Indicators
Revenue growth has exceeded expectations by 15% compared to the previous quarter, driven primarily by strong performance in the technology and healthcare sectors.

Market penetration in emerging markets has shown significant improvement, with particular strength in the Asia-Pacific region contributing to overall portfolio diversification.

## Strategic Recommendations
Implementation of digital transformation initiatives should be prioritized to maintain competitive advantage and improve operational efficiency across all business units.

Investment in research and development capabilities will be crucial for sustaining long-term growth and innovation in core product offerings.

## Risk Assessment
Current market volatility presents both challenges and opportunities that require careful monitoring and adaptive strategy implementation.

Supply chain dependencies need to be evaluated and diversified to mitigate potential disruptions in the global logistics network.

## Financial Outlook
Projected revenue targets remain achievable based on current market trends and pipeline analysis, with conservative estimates suggesting continued growth trajectory.

Cost optimization initiatives are expected to improve margins while maintaining quality standards and customer satisfaction levels.

## Conclusion
The strategic position remains strong with multiple growth opportunities identified for the next planning period, requiring focused execution and resource allocation.
"""

# Usage example
if __name__ == "__main__":
    generator = SummaryContentGenerator("template.pptx")
    
    try:
        generator = SimplifiedContentGenerator("template.pptx")
        generator.generate(sample_md_content, "output.pptx")
        print("Summary generation completed successfully!")
    except Exception as e:
        logging.error(f"Generation failed: {str(e)}")
